In [1]:
import json
from web3 import Web3
import web3 as w3
import numpy as np
import pandas as pd

In [2]:
ganache_URL = 'HTTP://127.0.0.1:7545'
web3 = Web3(Web3.HTTPProvider(ganache_URL))

In [3]:
# Set the default account 
web3.eth.defaultAccount = web3.eth.accounts[0]

In [4]:
# Create a dataset with all citizens 
# Suppose 5 citizens, 2 trucks and 2 plants 
assert len(web3.eth.accounts) == 10 

In [8]:
json_file = ''.join([i.strip() for i in open('abi.txt', 'r')])
abiRemix = json.loads(json_file)
addressRemix = web3.toChecksumAddress("0x8649045F12c67ab756cA5180310cA062953b9db4")
contract = web3.eth.contract(address=addressRemix, abi=abiRemix)

In [9]:
contract.all_functions()

[<Function createCitizen(address,string,uint256,uint256,uint256)>,
 <Function createStation(address,bool,uint256,uint256)>,
 <Function createTruck(address,bool)>,
 <Function deleteCitizen(address)>,
 <Function deleteStation(address)>,
 <Function deleteTruck(address)>,
 <Function renounceOwnership()>,
 <Function transferOwnership(address)>,
 <Function citizens(address)>,
 <Function numberC()>,
 <Function numberS()>,
 <Function numberT()>,
 <Function owner()>,
 <Function stations(address)>,
 <Function trucks(address)>]

In [10]:
print(contract.functions.owner().call())
print(contract.functions.numberC().call())
print(contract.functions.numberT().call())
print(contract.functions.numberS().call())
print(contract.functions.citizens(web3.eth.accounts[0]).call())

0x0c96C80F8cAb89FfF1454C8a35e5Accc70e2acc9
0
0
0
['0x0000000000000000000000000000000000000000', '', 0, 0, 0, False, 0, 0, False]


In [11]:
# Upload data from Municipality 
data = pd.read_excel('citizen_data.xlsx', engine="openpyxl")
data['address'] = web3.eth.accounts
assert contract.functions.owner().call() == data.loc[0,'address']

In [12]:
data

,name,surname,family,mq,weight,recycle,long,lat,role,address
0,Comune,Codogno,0.0,0.0,0.0,NaN,NaN,NaN,municipality,0x0c96C80F8cAb89FfF1454C8a35e5Accc70e2acc9
1,Francesca,Bianchessi,2.0,70.0,0.0,NaN,NaN,NaN,citizen,0x89c28b64Eb9d6666F6167Abca2Ca35396e48a85D
2,Ilaria,Bolla,4.0,200.0,0.0,NaN,NaN,NaN,citizen,0xaa371DB2Ddf15d5E26D431Eb9dD328036E9C169a
3,Alessandro,Botti,5.0,340.0,0.0,NaN,NaN,NaN,citizen,0x349928fAaC29c1E080BDCeDBc34c6971CA2DE882
4,Davide,Castellini,3.0,50.0,0.0,NaN,NaN,NaN,citizen,0x311230D0953791726a66EB7b55ABB9944a3CFd5e
5,Anna,Di Marco,4.0,80.0,0.0,NaN,NaN,NaN,citizen,0x94586aFEa2bce829483D63c0E285ceb9C4ca8A57
6,Truck,Recycle,NaN,NaN,NaN,True,NaN,NaN,truck,0xde72B6c2A1efF5098fb9A76e986732C8Fcaf71C2
7,Truck,Not Recycle,NaN,NaN,NaN,False,NaN,NaN,truck,0x27bC93859846067aB0472D8726F93014Ae3c8381
8,Disposal Station,Recycle,NaN,NaN,NaN,True,12.0,45.0,station,0x6df20f1bE2D87E08c9d3c797061e85971f68caDa
9,Disposal Station,Not Recycle,NaN,NaN,NaN,False,38.9,45.7,station,0xf5a673eeb7b40524900194309d49a093CCFB1504


In [ ]:
# Create CITIZENS - (address payable _address, string memory _name, uint _family, uint _house, uint256 _w)
citizensL = [[r.address, " ".join([r.name, r.surname]), int(r.family), int(r.mq), int(r.weight)] 
             for r in data.itertuples() if r.role == 'citizen']

# Create TRUCKS - (address _address, bool _recycle)
trucksL = [[r.address, r.recycle] for r in data.itertuples() if r.role == 'truck']

# Create STATIONS - (address _address, bool _recycle, uint _long, uint _lat)
stationsL = [[r.address, r.recycle, int(r.lat*(10**13)), int(r.long*(10**13))] for r in data.itertuples() if r.role == 'station']

In [ ]:
def create(c, t, s):
    for i in range(len(c)):
        contract.functions.createCitizen(c[i][0], c[i][1], c[i][2], c[i][3], c[i][4]).transact()
    for j in range(len(t)):
        contract.functions.createTruck(t[j][0], t[j][1]).transact()
    for k in range(len(s)):
        contract.functions.createStation(s[k][0], s[k][1], s[k][2], s[k][3]).transact()
    print('Done')

In [ ]:
create(citizensL, trucksL, stationsL)

In [ ]:
print(contract.functions.numberC().call())
print(contract.functions.numberT().call())
print(contract.functions.numberS().call())